In [9]:
import pandas as pd
import numpy as np
import plotly

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [15]:
#load data into dataframe
filepath = '../data/raw_data.csv'
raw_df = pd.read_csv(filepath)

Feature engineering brainstorm - underlined will be included in feature selection

1. advance to avg income ratio
2. advance to avg balance ratio
3. income stability proxy - min monthly income/ max monthly income
4. <u>balance volitility index</u> - min bal / avg bal (ideally we would have max bal too)
5. proportion of low balance days - captured in balance above 100
6. net cash flow 60D - already net assets
7. <u>deposit to withdrawl ratio</u> - avg spend/avg income
8. is first advance
9. <u>overdrafts per 30D</u> - overdraft count/(days open/30)
10. advance request hour - 



Notes:
- advance features dont mean anything since the field is uniform
- most features listed here may be valuable but would need access to unaggregrated data





### Approach
1. Opted to not floor/cap any potential outliers for simplicity
2. kept new features to only 3
3. While the paycheck model used is likely not a relevant feature decided to include and see how model uses it
4. Decided to not scale any features since we will be using XGBoost, though would likely create additional variables and try different scaling approaches to help normalize some of the larger scaled features

This could have also been created as a scikit learn pipeline but seemed overkill for simple operations

In [29]:

def new_features(df):
    """
    Adds proposed new features to raw input dataframe

    Args:
        df (pd.DataFrame): The input DataFrame

    Returns:
        pd.DataFrame: Expanded dataframe with new features
    """
    df['bal_vol_index'] = df['BalanceMin']/(df['BalanceAverage'] + .000001)

    df['dep_wd_ratio'] = df['AverageMonthlySpend']/(df['AverageMonthlyIncome'] + .000001)

    df['od_per_30d'] = df['OverdraftCount']/(df['TotalHistoryInDays']/30)

    return df


In [31]:
def imputation(df):
    """
    Imputes missing values with zero and adds a missing indicator column for each column
    that had missing values.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed and indicator columns added.
    """
    df_processed = df.copy()
    for col in df_processed.columns:
        if df_processed[col].isnull().any():
            df_processed[f'{col}_missing'] = df_processed[col].isnull().astype(int)
            df_processed[col] = df_processed[col].fillna(0)
    return df_processed


In [43]:
def one_hot_encode_paycheck_model(df):
    """
    One-hot encodes the 'PaycheckModelUsed' column in the raw data.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame with the 'PaycheckModelUsed' column.

    Returns:
    pd.DataFrame: A new DataFrame with one-hot encoded 'PaycheckModelUsed' columns.
    """
    return pd.get_dummies(df, 
                          columns=["PaycheckModelUsed"], 
                          prefix="PaycheckModel", 
                          drop_first=True,
                          dtype='int64')

In [45]:
def preprocess(df):
    """
    Runs full data preparation pipeline
    """
    df_new = new_features(df)

    df_imputed = imputation(df_new)

    df_final = one_hot_encode_paycheck_model(df_imputed)

    return df_final

In [47]:
df_final = preprocess(raw_df)

In [49]:
df_final.head()

,DefaultedAdvances,AdvanceAmount,CurrentBalance,LatefeesTotalCount,CreditAccounts,LastRepaymentAmount,OverdraftCount,OverdraftTotal,AverageMonthlySpend,BalanceAverage,BalanceMin,TotalAssets,AverageNumberOfTransactionsADay,TotalCash,LatefeesCount,Paycheck,TotalHistoryInDays,AverageMonthlyIncome,AverageMonthlyDiscretionarySpend,CheckingAccountCount,CreditAccountCount,SavingsAccountCount,OutstandingCreditDebtWherePayingInterest,AverageNumberOfTransactionsADayPrimaryChecking,NegativeBalanceCount,NumberOfMatches,ErrorRate,IsNameBased,BalanceAbove100L30Count,AveragePotentialMonthlyIncome,HasEmpowerBanking,bal_vol_index,dep_wd_ratio,od_per_30d,CurrentBalance_missing,PaycheckModelUsed_missing,NumberOfMatches_missing,ErrorRate_missing,IsNameBased_missing,bal_vol_index_missing,PaycheckModel_BruteForce,PaycheckModel_DeepSearch,PaycheckModel_Tagging,PaycheckModel_VariableIncomeInMonth
0,0,50,268.10,0,0,0.0,0,0,-5491.060000,345.11,-185.34,268.10,4.633333,268.10,0,2457.000000,61.179873,3969.461181,-1191.270000,1,0,1,0.0,4.633333,11,5.0,0.0,1.0,18,4398.506667,1,-0.537046,-1.383326,0.0,0,0,0,0,0,0,1,0,0,0
1,0,50,17.68,0,0,0.0,0,0,-1330.759925,378.01,0.08,17.72,1.647577,17.72,0,1706.900000,128.706924,1120.506790,-1503.200000,1,0,2,0.0,4.763780,0,7.0,0.0,0.0,28,3436.180000,0,0.000212,-1.187641,0.0,0,0,0,0,0,0,1,0,0,0
2,0,50,0.79,0,0,0.0,0,0,-6446.753876,559.13,-276.61,273.99,4.080935,273.99,0,4630.347222,558.401660,6674.567286,-1546.523333,1,0,1,0.0,3.562950,20,5.0,0.0,1.0,16,6803.280000,1,-0.494715,-0.965868,0.0,0,0,0,0,0,0,1,0,0,0
3,0,50,-12.83,0,0,0.0,0,0,-1904.714237,124.86,-44.62,645.12,2.831579,645.12,0,1300.000000,285.776868,1787.241711,-940.616667,1,0,1,0.0,2.729825,15,3.0,0.0,0.0,15,0.000000,0,-0.357360,-1.065728,0.0,0,0,0,0,0,0,1,0,0,0
4,0,50,55.88,0,0,0.0,0,0,-5857.241057,420.26,42.50,55.88,8.926829,55.88,0,3633.073333,165.990591,5169.470216,-1893.433333,1,0,0,0.0,8.926829,0,3.0,0.0,0.0,24,7390.746667,1,0.101128,-1.133045,0.0,0,0,0,0,0,0,1,0,0,0


In [51]:
df_final.to_csv('../data/final.csv')